# **Signa_Lab ITESO:** Generador de *Embbeddings*

## **Cuaderno 02:** Generación de *embeddings*, reducción de dimensionalidades y clusterización.

Cuaderno de código para generar *embeddings* (relaciones semánticas codificadas en vectores) a partir de datos textuales, idealmente, procesados y depurados con antelación ([ver cuaderno 01](https://github.com/signalab/generador-embeddings/blob/main/cuadernos/01_Signa_Lab_generador_embeddings_Depuraci%C3%B3n_importar_limpiar_depurar_texto_01.ipynb)), con ayuda de modelos de lenguaje de la librería [sentence-transformers](https://www.sbert.net/), alojados en repositorios de [HuggingFace](https://huggingface.co/sentence-transformers) (en la nube) o descargados localmente.

## 1. Importar librerías, dependencias y archivo de datos a para la generación de *embeddings*


**Instalar librerías:**

In [ ]:
# Instalar librerías de Python necesarias

!pip install tensorflow_text
!pip install pandas
!pip install sentence_transformers
!pip install numpy
!pip install operator
!pip install torch
!pip install tqdm
!pip install transformers
!pip install tensorflow

!pip install matplotlib
!pip install scikit-learn
!pip install plotly
!pip install umap-learn
!pip install yellowbrick

**Importar librerías:**

In [ ]:
# Importar librerías de Python necesarias

import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
import operator
from tqdm import tqdm
import time
import torch
from transformers import BertForMaskedLM, BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf


import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import plotly.express as px
from sklearn.decomposition import PCA
import umap
from sklearn.manifold import TSNE
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramCollocationFinder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn import datasets
from yellowbrick.cluster import SilhouetteVisualizer

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Definir función para implementar barra de progreso sobre el procesamiento
def with_progress_bar(func):
    def wrapper(*args, **kwargs):
        # Inicia el contador de tiempo
        start_time = time.time()

        # Crea una barra de progreso
        pbar = tqdm(total=100, desc="Processing", ncols=70)

        # Define una función interna para actualizar la barra de progreso
        def update_progress_bar(i):
            pbar.update(i)

        # Agrega la función de actualización al diccionario de argumentos
        kwargs['update_progress_bar'] = update_progress_bar

        # Ejecuta la función original
        result = func(*args, **kwargs)

        # Finaliza la barra de progreso
        pbar.close()

        # Calcula y muestra el tiempo de ejecución
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Tiempo de ejecución: {execution_time:.2f} segundos")

        return result
    return wrapper

**Cargar modelo de lenguaje** para su implementación en librería de *sentence-transformers*:

In [ ]:
# Indicar ruta y nombre de modelo de lenguaje a cargar en librería de sentence transformers
# puedes explorar los modelos incorporados a librería de sentence-transformers en su repositorio en HuggingFace: https://huggingface.co/sentence-transformers

# ejemplo de ruta de modelo en la nube cargado desde Hugging Face:
modelo = "intfloat/multilingual-e5-large-instruct"

# ejemplo de ruta de modelo cargado localmente:
# modelo = "./modelos/multilingual-e5-large-instruct"

# cargar modelo para embeddings
embedder = SentenceTransformer(modelo)

In [ ]:
embedder

**Indicar rutas y cargar archivos de datos** para generar *embeddings*:

In [ ]:
# PARA UN SOLO ARCHIVO (generar embeddings de un archivo individual):
# especificar ruta y formato de archivo a importar:
# ruta = "./archivo-datos-para-embeddings.csv" # importar CSV
# ruta = "./archivo-datos-para-embeddings.xlsx" # importar Excel


# extraer nombre de archivo como referencia
# nombreArchivo = ruta.split("/")[-1].split('.')[0]

# crear data frame con archivo de datos importado
# df = pd.read_csv(ruta)  # data frame desde CSV
# df = pd.read_excel(ruta)  # data frame desde Excel



# PARA MÚLTIPLES ARCHIVOS (generar embeddings para cada uno):
# especificar rutas y formatos de distintos archivos a importar:
ruta1 = "./archivo-datos-para-embeddings1.csv" # importar CSV
ruta2 = "./archivo-datos-para-embeddings2.csv" # importar CSV
ruta3 = "./archivo-datos-para-embeddings3.csv" # importar CSV

# ruta1 = "./archivo-datos-para-embeddings1.xlsx" # importar Excel
# ruta2 = "./archivo-datos-para-embeddings2.xlsx" # importar Excel
# ruta3 = "./archivo-datos-para-embeddings3.xlsx" # importar Excel

# crear data frames con archivos de datos importados
df1 = pd.read_csv(ruta1)  # data frame desde CSV
df2 = pd.read_csv(ruta2)  # data frame desde CSV
df3 = pd.read_csv(ruta3)  # data frame desde CSV

# df1 = pd.read_excel(ruta1)  # data frame desde Excel
# df2 = pd.read_excel(ruta2)  # data frame desde Excel
# df3 = pd.read_excel(ruta3)  # data frame desde Excel


# extraer nombres de archivos como referencia
nombreArchivo1 = ruta1.split("/")[-1].split('.')[0]
nombreArchivo2 = ruta2.split("/")[-1].split('.')[0]
nombreArchivo3 = ruta3.split("/")[-1].split('.')[0]

**Revisar datos importados:**

**Para un solo archivo**, revisar registros importados desde archivo individual:

In [ ]:
# Previsualizar tabla de datos importados de archivo individual
# df.head()

In [ ]:
# Verificar número de filas y columnas en datos importados de archivo individual
# df.shape

**Para múltiples archivos**, revisar registros importados desde cada ruta:

In [ ]:
# Previsualizar tabla de datos importados de 1er archivo (ruta1)
df1.head()

In [ ]:
# Verificar número de filas y columnas en datos importados de 1er archivo (ruta1)
df1.shape

In [ ]:
# Previsualizar tabla de datos importados de 2ndo archivo (ruta2)
df2.head()

In [ ]:
# Verificar número de filas y columnas en datos importados de 2ndo archivo (ruta2)
df2.shape

In [ ]:
# Previsualizar tabla de datos importados de 3er archivo (ruta3)
df3.head()

In [ ]:
# Verificar número de filas y columnas en datos importados de 3er archivo (ruta3)
df3.shape

## 2. Generar *embeddings* con el modelo de lenguaje cargado

**Definir función para generar embeddings:**

In [ ]:
# Definir función para generar embeddings
@with_progress_bar
def generate(mod, dfl, colText, update_progress_bar=None):
    tot = len(dfl)
    dfW = dfl
    dfW.loc[:, "Embedding"] = None
    for index, row in dfW.iterrows():
        # Iterar sobre el DataFrame y codificar cada texto
        embedding = mod.encode(str(row[colText])).tolist()
        dfW.at[index, 'Embedding'] = embedding
        # Actualizar la barra de progreso una vez por cada 10 filas
        if update_progress_bar is not None:
            update_progress_bar((index/tot)*2)
    return dfW

**Para un solo archivo**, generar embeddings con archivo de datos cargado en data frame:

In [ ]:
# Generar embeddings de un solo archivo

# Indicar columna de data frame con texto a procesar
# textCol = "sem_text"
# Ejecutar generación de embeddings con datos y modelo cargados
# datasetEmbeddings = generate(embedder, df, textCol)

**Para múltiples archivos**, generar embeddings por cada uno de los archivos de datos cargados en sus respectivos data frames:

In [ ]:
# Generar embeddings de 1er data frame (df1)

# Indicar columna de data frame con texto a procesar
textCol1 = "sem_text"
# Ejecutar generación de embeddings con datos y modelo cargados
datasetEmbeddings1 = generate(embedder, df1, textCol1)

In [ ]:
# Generar embeddings de 2ndo data frame (df2)

# Indicar columna de data frame con texto a procesar
textCol2 = "sem_text"
# Ejecutar generación de embeddings con datos y modelo cargados
datasetEmbeddings2 = generate(embedder, df2, textCol2)

In [ ]:
# Generar embeddings de 2ndo data frame (df2)

# Indicar columna de data frame con texto a procesar
textCol3 = "sem_text"
# Ejecutar generación de embeddings con datos y modelo cargados
datasetEmbeddings3 = generate(embedder, df3, textCol3)

**Exportar archivos de datos con embeddings generados (en formato CSV):**

In [ ]:
# Exportar archivos de datos con embeddings en CSV:


# Desde un solo archivo, exportar CSV con embeddings procesados:
# df.to_csv(f"{nombreArchivo}Embeddings.csv")
# print(f"{nombreArchivo}Embeddings descargado en csv")


# Desde múltiples archivos, exportar CSVs con embeddings procesados por cada uno:
df1.to_csv(f"{nombreArchivo1}_Embeddings.csv")
df2.to_csv(f"{nombreArchivo2}_Embeddings.csv")
df3.to_csv(f"{nombreArchivo3}_Embeddings.csv")


print(f"{nombreArchivo1}_Embeddings descargado en csv")
print(f"{nombreArchivo2}_Embeddings descargado en csv")
print(f"{nombreArchivo3}_Embeddings descargado en csv")

## **3. Reducción de dimensiones con distintas técnicas (UMAP, PCA, TSNE)**

**Elegir data frame** con embeddings para reducir dimensionalidaes y clusterizar:

In [ ]:
# Elegir data frame con embeddings (df, en caso de un solo archivo, df1,2,3... e caso de múltiples archivos:

# Un solo archivo cargado con embeddings generados
# dfClusters = df

# Para múltiples archivos, elegir número de data frame y repetir proceso
dfClusters = df1
# dfClusters = df2
# dfClusters = df3

Definir función para reducir dimensionalidades con: **UMAP**

In [ ]:
# Definir función para UMAP
def genUMAP(df, columnaEmbeddings, nDims):
    dfW = df
    umap_model = umap.UMAP(n_components=nDims)
    X_umap = umap_model.fit_transform(dfW[columnaEmbeddings].tolist())
    dfW["embeddingsReducidos"] = X_umap.tolist()
    return dfW

Definir función para reducir dimensionalidades con: **PCA**

In [ ]:
# Definir función para PCA
def genPCA(df, columnaEmbeddings, nDims):
    dfW = df
    pca = PCA(n_components=nDims)
    X_pca = pca.fit_transform(dfW[columnaEmbeddings].tolist())
    dfW["embeddingsReducidos"] = list(X_pca)
    return dfW

Definir función para reducir dimensionalidades con: **TSNE**

In [ ]:
# Definir función para TSNE
def genTSNE(df, columnaEmbeddings, nDims):
    dfW = df
    tsne = TSNE(n_components=nDims, random_state=42)
    X_tsne = tsne.fit_transform(dfW[columnaEmbeddings].tolist())
    dfW["embeddingsReducidos"] = list(X_tsne)
    return dfW

**Definir función para elegir método para reducir dimensiones:**

In [ ]:
def reduceDim(df, columnaEmbeddings, nDims, method):
    if method.lower() == 'umap':
        return genUMAP(df, columnaEmbeddings, nDims)
    elif method.lower() == 'pca':
        return genPCA(df, columnaEmbeddings, nDims)
    elif method.lower() == 'tsne':
        return genTSNE(df, columnaEmbeddings, nDims)

**Elegir método a utilizar, número de dimensiones (2D o 3D) y ejecutar reducción de dimensionalidades sobre *embeddings* cargados:**

In [ ]:
#Indicar nombre de columna con embeddings completos (vector dado por las dimensiones del modelo de lenguaje)
columnaConEmbeddings = "Embedding"

#Indicar si se busca reducir a 2 o 3 dimensiones para visualizador 2D o 3D
numDimensiones = 3

# Elegir método para reducción de dimensionalidades: "umap", "tsne", "pca"
metodo = "umap"

# Ejecutar reducción de dimensionalidades
datasetConDimensionesReducidas = reduceDim(dfClusters, columnaConEmbeddings, numDimensiones, metodo)

In [ ]:
datasetConDimensionesReducidas.head()

## **4. Clusterización y visualización de *embeddings***

Definir función para **Método del Codo** para calcular número ideal de clústers para los datos cargados:

In [ ]:
def elbowMethod(df, columnWiEmbeddings, rangoPosibleDeCluster, byRegion=False):
    # Convierte la columna de embeddings del DataFrame a un array de numpy para su procesamiento
    embeddings = np.array(df[columnWiEmbeddings].tolist())
    # Calcula la inercia para diferentes valores de k (número de clústeres)
    # Inicializa una lista vacía para almacenar los valores de inercia calculados para cada número de clústeres
    inertia_values = []
    # Define un rango de valores de k (número de clústeres) para probar, basándose en el rango proporcionado como argumento
    possible_k_values = range(rangoPosibleDeCluster[0], rangoPosibleDeCluster[1]+1)  # Prueba k desde n1 hasta n2 clústeres

    # Itera sobre el rango de posibles valores de k
    for index, k in enumerate(possible_k_values):
        # Inicializa el algoritmo KMeans con el número actual de clústeres (k) y una semilla aleatoria fija
        kmeans = KMeans(n_clusters=k, random_state=0)

        # Entrena el modelo KMeans con los embeddings
        kmeans.fit(embeddings)

        # Añade la inercia del modelo (la suma de las distancias cuadradas de las muestras a su centro de clúster más cercano) a la lista de valores de inercia
        inertia_values.append(kmeans.inertia_)

    # Prepara los resultados para ser graficados, conteniendo los valores de k y las inercias correspondientes
    resToPlot = [possible_k_values, inertia_values]

    # Grafica el método del codo utilizando matplotlib, donde el eje X representa el número de clústeres y el eje Y la inercia
    plt.plot(resToPlot[0], resToPlot[1], marker='o')
    plt.xlabel('Número de Clústeres (k)')
    plt.ylabel('Inercia')
    plt.title('Método del Codo para Determinar k')
    plt.show()

**Ejecutar método del codo** para identificar número deseable de clústers:

In [ ]:
# Indicar nombre de columna con embeddings reducidos
columnaConEmbeddings = "embeddingsReducidos"
# Calcular número ideal en un rango de 1 a 20 clústers
rangoDeClusters = [1,20]
elbowMethod(datasetConDimensionesReducidas, columnaConEmbeddings, rangoDeClusters)

Definir función para **Método de Silueta** para calcular número ideal de clústers para los datos cargados

In [ ]:
def silhouetteMethod(df, columnWithEmbeddings, rangeOfClusters):
    # Convierte la columna de embeddings del DataFrame a un array de numpy para su procesamiento
    embeddings = np.array(df[columnWithEmbeddings].tolist())

    # Establece el número de subplots basado en el rango de clústeres a analizar
    n_clusters = range(rangeOfClusters[0], rangeOfClusters[1] + 1)
    n_rows = len(n_clusters) // 2 + len(n_clusters) % 2
    fig, ax = plt.subplots(n_rows, 2, figsize=(16, 32))
    plt.subplots_adjust(hspace=0.5)

    # Itera sobre el rango especificado de número de clústeres
    for i, k in enumerate(n_clusters):
        # Crea una instancia de KMeans para el número actual de clústeres
        km = KMeans(n_clusters=k, init='k-means++', n_init=10, max_iter=300, random_state=42)

        # Calcula la fila y columna para el subplot actual
        q, mod = divmod(i, 2)

        # Crea una instancia de SilhouetteVisualizer para visualizar el coeficiente de silueta
        visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q][mod])

        # Ajusta el visualizador con los embeddings
        visualizer.fit(embeddings)

        # Establece el título del subplot
        ax[q][mod].set_title(f'Clusters: {k}')

    # Ajusta la disposición de los subplots para evitar solapamientos
    plt.tight_layout()
    plt.show()

In [ ]:
# Ejecutar función de método de silueta
silhouetteMethod(dfClusters, 'embeddingsReducidos', (2, 20))

Definir función de clusterizarización utilizando método **K-Means:**

In [ ]:
# Definir función de clusterización con K-means
def clusterKmeans(df, optimal_k_value, columnaEmbeddings, byRegion=False, cluster_col_name=None):
    embeddings = df[columnaEmbeddings].tolist()
    dfW = df.copy() # Crear copia de trabajo de DataFrame

    if cluster_col_name is None:
      cluster_col_name = "cluster"

    try:
        if byRegion: # Clusterización por región (opcional, no utilizado en metodología final)
            # Inicializar nueva columna para alojar etiquetas de clústeres
            dfW[cluster_col_name] = None

            # Iterar sobre cada región (opcional)
            for region, df_region in dfW.groupby('region'):
                # Aplicar clusterización de K-means en cada región (opcional)
                kmeans = KMeans(n_clusters=optimal_k_value, random_state=0)
                cluster_labels = kmeans.fit_predict(df_region[columnaEmbeddings].tolist())

                # Añadir prefijo de clúster por región (opcional)
                prefixed_cluster_labels = [f"{region}_{label}" for label in cluster_labels]

                # Asignar etiquetas de clúster por región a DataFrame
                dfW.loc[dfW['region'] == region, cluster_col_name] = prefixed_cluster_labels
        else:
            # Aplicar clusterización por K-means a todo el conjunto de datos importado (versión implementada en metodología final)
            kmeans = KMeans(n_clusters=optimal_k_value, random_state=0)
            cluster_labels = kmeans.fit_predict(embeddings)
            dfW[cluster_col_name] = cluster_labels

        return dfW
    except Exception as e:
        print(f"Error al generar clusters: {e}")
        return pd.DataFrame({0: ["Error al generar clusters"]})


**Ejecutar clusterización** con número deseado de clúster con método *k-means:*

In [ ]:
# Indicar nombre de columna con embeddings reducidos
columnaConEmbeddings = "embeddingsReducidos"
# Indicar número de clústers a segmentar
numeroDeClusters = 7
dfClusterizado = clusterKmeans(datasetConDimensionesReducidas, numeroDeClusters, columnaConEmbeddings, byRegion=False)

In [ ]:
# Previsualizar tabla de registros con sus respectivos clústers asignados
dfClusterizado

Definir función para **visualizar relaciones semánticas y clústeres en 3D:**

In [ ]:
def visualize3D(df, columnaEmbeddingsReducidas, columnaTexto, byRegion=False, columnaCluster=None):
  if columnaCluster is None:
    columnaCluster = "cluster"
  embeddingWithSelectedDimensions = df[columnaEmbeddingsReducidas].tolist()
  dfToPlot = {
        "X": [x[0] for x in embeddingWithSelectedDimensions],
        "Y": [y[1] for y in embeddingWithSelectedDimensions],
        "Z": [z[2] for z in embeddingWithSelectedDimensions],
        "cluster": df[columnaCluster].tolist(),
        "text": df[columnaTexto]
  }

  fig = px.scatter_3d(dfToPlot, x='X', y='Y', z='Z', color='cluster', hover_data=["text"],
                         labels={'X': 'Dimensión 1', 'Y': 'Dimensión 2', 'Z': 'Dimensión 3'},
                         title=f'Visualización de clústers semánticos',
                         color_discrete_sequence=px.colors.sequential.Viridis)
  fig.show()

**Ejecutar visualización de clústers 3D:**

In [ ]:
# Indicar nombre de columna con embeddings reducidos
columnaConEmbeddings = "embeddingsReducidos"
# Indicar columna de texto para etiquetas
columnaTextoClusterizado = "clean_text"
visualize3D(dfClusterizado, columnaConEmbeddings, columnaTextoClusterizado, columnaCluster="cluster")

**Exportar archivos de datos (formato CSV y JSON) con clústers calculados:**

In [ ]:
# Exportar en json, haciendo referencia al data frame elegido
dfClusterizado.to_json(f"{nombreArchivo1}_Clusters.json")

In [ ]:
# Exportar en csv, haciendo referencia al data frame elegido
dfClusterizado.to_csv(f"{nombreArchivo1}_Clusters.csv")

### **(POSTPROCESAMIENTO PROVISIONAL PARA COSMOGRAPH):**

**Receta de OpenRefine** para preparar dataset clusterizado para su visualización en **Cosmograph**:

- Renombrar columna con identificador único id a **id**, en caso de que no exista.
- Crear columna **cluster-str** basada en cluster. Cambiar valores a letras con faceta de texto o con el script de python siguiente:


```
# Define a mapping from numbers to letters
number_to_letter = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E',
    5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
    10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O',
    15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T'
}

# The value of the cell is stored in the variable 'value'
# Convert value to integer, if it's a string representation of a number
# and then get the corresponding letter, if the number is between 0 and 19
try:
    # Assuming 'value' is the cell value, make sure it is an integer
    number = int(value)
    # Return the corresponding letter if the number is between 0 and 19
    if 0 <= number <= 19:
        return number_to_letter[number]
    else:
        return value  # Return the original value if not in the specified range
except ValueError:
    # Return the original value if it is not a number
    return value
```


- Crear nueva columna **x**  a partir de reduced_embeddings. Aplicar transformación de celda con el siguiente script de Python:


```
import ast

# Convert the string to a list using ast.literal_eval()
value_list = ast.literal_eval(value)

# Get the first element of the list
x = value_list[0]

return x
```


- Crear nueva columna **y** a partir de reduced_embeddings. Aplicar transformación de celda con el siguiente script de Python:



```
import ast

# Convert the string to a list using ast.literal_eval()
value_list = ast.literal_eval(value)

# Get the first element of the list
y = value_list[1]

return y
```

## **5. Referencias:**

* Bird, Steven, Edward Loper & Ewan Klein (2009). Natural Language Processing with Python.  O'Reilly Media Inc.
* McInnes, L., Healy, J., & Melville, J. (2018). Umap: Uniform manifold approximation and projection for dimension reduction. arXiv preprint arXiv:1802.03426. https://arxiv.org/abs/1802.03426
* Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O., Blondel, M., Prettenhofer, P., Weiss, R., Dubourg, V., Vanderplas, J., Passos, A., Cournapeau, D., Brucher, M., Perrot, M., Duchesnay, E., & others. (2011). Scikit-learn: Machine Learning in Python. Journal of Machine Learning Research, 12, 2825–2830.
* Reimers, N., & Gurevych, I. (2019). Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks (arXiv:1908.10084). arXiv. http://arxiv.org/abs/1908.10084
* Rousseeuw, P. (1987). Silhouettes: a Graphical Aid to the Interpretation and Validation of Cluster Analysis. Computational and Applied Mathematics. 20: 53–65. doi:10.1016/0377-0427(87)90125-7.
* Spärck-Jones, K. (1972). A statistical interpretation of term specificity and its application in retrieval. Journal of Documentation, 28(1), 11-21. https://www.staff.city.ac.uk/~sbrp622/idfpapers/ksj_orig.pdf
* Thorndike, R. (1953). Who Belongs in the Family?. Psychometrika. 18 (4): 267–276. doi:10.1007/BF02289263. S2CID 120467216.
* Wang, L., Yang, N., Huang, X., Yang, L., Majumder, R., & Wei, F. (2024). Multilingual E5 Text Embeddings: A Technical Report. arXiv preprint arXiv:2402.05672. Recuperado de https://arxiv.org/abs/2402.05672


## **6. Créditos**

**Realizado por el equipo de Signa_Lab ITESO:**

- **Programación de cuadernos de código (Python)**:
Javier de la Torre Silva, José Luis Almendarez González y Diego Arredondo Ortiz

- **Supervisión del desarrollo tecnológico y documentación:**
Diego Arredondo Ortiz

- **Equipo de Coordinación Signa_Lab ITESO:**
Paloma López Portillo Vázquez, Víctor Hugo Ábrego Molina y Eduardo G. de Quevedo Sánchez

Mayo, 2024. Instituto Tecnológico y de Estudios Superiores de Occidente (ITESO)
Tlaquepaque, Jalisco, México.
